# Do stricter gun laws reduce firearm gun deaths?

In [30]:
import requests
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import folium

### Datasources

We can use the below NCHS API endpoint to see cause of death rates per state per quarter in the US. This will be a good starting point as we look to visualize per state



In [31]:
# Grabbing data from API endpoint
response = requests.get("https://data.cdc.gov/resource/489q-934x.json").json()

In [32]:
# Write response to pandas dataframe
df = pd.json_normalize(response)
df.head()

,year_and_quarter,time_period,cause_of_death,rate_type,unit,rate_overall,rate_sex_female,rate_sex_male,rate_alaska,rate_alabama,...,rate_age_1_4,rate_age_5_14,rate_age_15_24,rate_age_25_34,rate_age_35_44,rate_age_45_54,rate_age_55_64,rate_65_74,rate_age_75_84,rate_age_85_plus
0,2022 Q1,12 months ending with quarter,All causes,Age-adjusted,"Deaths per 100,000",873.2,729.4,1038,944.5,1109.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022 Q1,12 months ending with quarter,Alzheimer disease,Age-adjusted,"Deaths per 100,000",30.6,35,23.8,28.5,45.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022 Q1,12 months ending with quarter,COVID-19,Age-adjusted,"Deaths per 100,000",95,75.2,119.1,121.3,133.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022 Q1,12 months ending with quarter,Cancer,Age-adjusted,"Deaths per 100,000",145.9,127.4,170.9,156,159.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022 Q1,12 months ending with quarter,Chronic liver disease and cirrhosis,Age-adjusted,"Deaths per 100,000",14.4,10.3,18.9,25.5,16.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1. Filter for Firearm-Related Deaths:

In [33]:
# Filter the DataFrame for firearm-related deaths
df_firearm = df[df['cause_of_death'].str.contains('Firearm', case=False, na=False)]


2. Categorize Gun Laws:

In [34]:
# Example gun law strictness categories
gun_laws = {
    'California': 5,
    'Texas': 1,
    'New York': 4,
    'Alabama': 2,
    'Florida': 2,
    'Illinois': 4,
    # Add all other states with their appropriate scale
}

# Convert gun law dictionary into DataFrame
df_gun_laws = pd.DataFrame(list(gun_laws.items()), columns=['state', 'gun_law_strictness'])


3. Merge Firearm Mortality with Gun Law Data:

In [35]:
import pandas as pd

# Load the firearm mortality data
df_firearm = pd.read_csv('/content/data-table.csv')

# Display the merged dataframe
df_firearm.head()


,YEAR,STATE,RATE,DEATHS,URL
0,2022,AL,25.5,"1,278",/nchs/pressroom/states/alabama/al.htm
1,2022,AK,22.4,164,/nchs/pressroom/states/alaska/ak.htm
2,2022,AZ,20.1,"1,535",/nchs/pressroom/states/arizona/az.htm
3,2022,AR,21.9,666,/nchs/pressroom/states/arkansas/ar.htm
4,2022,CA,8.6,"3,484",/nchs/pressroom/states/california/ca.htm


In [36]:
# Load the firearm mortality data from the CSV file
df_firearm_mortality = pd.read_csv('/content/data-table.csv')

# Check the structure of the data
df_firearm_mortality.head()

# Rename the columns to match merging criteria if necessary
df_firearm_mortality.columns = ['year', 'state', 'rate', 'deaths', 'url']


In [37]:
df_firearm_mortality.head()

,year,state,rate,deaths,url
0,2022,AL,25.5,"1,278",/nchs/pressroom/states/alabama/al.htm
1,2022,AK,22.4,164,/nchs/pressroom/states/alaska/ak.htm
2,2022,AZ,20.1,"1,535",/nchs/pressroom/states/arizona/az.htm
3,2022,AR,21.9,666,/nchs/pressroom/states/arkansas/ar.htm
4,2022,CA,8.6,"3,484",/nchs/pressroom/states/california/ca.htm


Merge Firearm Mortality Data with Gun Law Strictness

In [39]:
# Merge firearm mortality data with gun law strictness data on 'state'
merged_df = df_firearm_mortality.merge(df_gun_laws, on='state', how='inner')

# Check the merged DataFrame
merged_df.head()


,year,state,rate,deaths,url,gun_law_strictness
